In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [ ]:
# from AirNowAPI documentation
# Outputs: https://docs.airnowapi.org/Data/docs
header_cols = ['lat', 'lon', 'utc', 'parameter','concentration', 'unit', 'raw_concentration',
    'aqi', 'aqi_category', 'site_name', 'site_agency', 'aqs_id', 'full_aqs_id']

In [32]:
df.head()

,lat,lon,utc,parameter,concentration,unit,raw_concentration,aqi,aqi_category,site_name,site_agency,aqs_id,full_aqs_id
0,37.7658,-122.3978,2018-09-01T00:00,PM2.5,6.2,UG/M3,6.0,26,1,San Francisco,San Francisco Bay Area AQMD,60750005,840060750005
1,37.7658,-122.3978,2018-09-01T00:00,NO2,3.1,PPB,3.1,3,1,San Francisco,San Francisco Bay Area AQMD,60750005,840060750005
2,37.7658,-122.3978,2018-09-01T00:00,OZONE,-999.0,PPB,18.0,-999,-999,San Francisco,San Francisco Bay Area AQMD,60750005,840060750005
3,37.7658,-122.3978,2018-09-01T00:00,CO,0.2,PPM,0.2,-999,-999,San Francisco,San Francisco Bay Area AQMD,60750005,840060750005
4,37.7658,-122.3978,2018-09-01T01:00,PM2.5,7.6,UG/M3,9.0,32,1,San Francisco,San Francisco Bay Area AQMD,60750005,840060750005


In [157]:
def clean_aqi_data(df:pd.DataFrame) -> pd.DataFrame:
    df['date'] = pd.to_datetime(df['utc']).dt.date
    return df

def aggregate_measures_data(df:pd.DataFrame) -> pd.DataFrame:
    df_agg = df.groupby(['date', 'parameter']) \
        .agg(max_daily_concentration = ('raw_concentration', 'max')) \
        .reset_index()
    return df_agg

def make_measures_agg_wide(df:pd.DataFrame) -> pd.DataFrame:
    df_wide = df.pivot(index='date', columns='parameter') \
    .reset_index()
    df_wide.columns = df_wide.columns.droplevel(0)
    df_wide  = df_wide.rename(columns={'':'date'})
    return df_wide

def make_quality_split(df):
    df_aqi = df[['date', 'aqi', 'aqi_category']]
    df_agg_aqi = df_aqi.groupby('date') \
        .agg(max_aqi = ('aqi', 'max'),
            max_aqi_category = ('aqi_category', 'max')) \
        .reset_index()
    return df_agg_aqi

def feed_aqi_files(file_path:str, header_cols: list, v:int) -> tuple[pd.DataFrame, list]:
    files =  os.listdir(file_path)
    df_combo = pd.DataFrame()
    error_log = []
    for i, f in enumerate(files):
        if v > 0 :
            print(f"PROCESS FILE {i}: {f}")
        try:
            df = pd.read_csv(file_path + f, names=header_cols)
        except:
            print(f"WARNING - could not read file {f}")
            error_log.append(f)
            continue
        df_clean = clean_aqi_data(df)
        df_agg_measures = aggregate_measures_data(df_clean)
        # print(f"df_agg_measures : \n {df_agg_measures.head()}")
        # print(f"df_agg_measures columns: {df_agg_measures.columns}")
        # print(f"missing parameter {sum(df_agg_measures['parameter'].isna())}")
        df_agg_measures_wide = make_measures_agg_wide(df_agg_measures)
        df_agg_aqi = make_quality_split(df_clean)
        df_agg = df_agg_aqi.merge(df_agg_measures_wide, how='left', on='date') 

        df_combo = pd.concat([df_combo, df_agg])
    
    return (df_combo, error_log)

In [153]:
df_clean = clean_aqi_data(df)
df_agg_measures = aggregate_measures_data(df_clean)

In [151]:
df_wide_test = df_agg_measures.pivot(index='date', columns='parameter') \
    .reset_index()

df_wide_test.columns = df_wide_test.columns.droplevel(0)
df_wide_test  = df_wide_test.rename(columns={'':'date'})
df_wide_test.head()

parameter,date,CO,NO2,OZONE,PM2.5
0,2018-09-01,0.3,5.0,22.0,19.0
1,2018-09-02,0.3,8.2,32.0,20.0
2,2018-09-03,0.3,5.1,35.0,24.0
3,2018-09-04,0.4,10.5,36.0,23.0
4,2018-09-05,0.4,11.3,36.0,23.0


In [158]:
file_path = '../../data/raw/air-quality/'
header_cols = ['lat', 'lon', 'utc', 'parameter','concentration', 'unit', 'raw_concentration',
    'aqi', 'aqi_category', 'site_name', 'site_agency', 'aqs_id', 'full_aqs_id']

df_daily_max, error_log = feed_aqi_files(file_path, header_cols=header_cols, v=1)

PROCESS FILE 0: aqi_download_2018-09-01_2018-12-31.csv
PROCESS FILE 1: aqi_download_2017-05-01_2017-08-31.csv
PROCESS FILE 2: aqi_download_2019-09-01_2019-12-31.csv
PROCESS FILE 3: .DS_Store
WARNING - could not read file .DS_Store
PROCESS FILE 4: aqi_download_2017-01-01_2017-04-30.csv
PROCESS FILE 5: aqi_download_2017-09-01_2017-12-31.csv
PROCESS FILE 6: aqi_download_2018-05-01_2018-08-31.csv
PROCESS FILE 7: aqi_download_2019-05-01_2019-08-31.csv
PROCESS FILE 8: aqi_download_2019-01-01_2019-04-30.csv
PROCESS FILE 9: aqi_download_2018-01-01_2018-04-30.csv


In [166]:
df_daily_max

,date,max_aqi,max_aqi_category,CO,NO2,OZONE,PM2.5
0,2018-09-01,52,2,0.3,5.0,22.0,19.0
1,2018-09-02,61,2,0.3,8.2,32.0,20.0
2,2018-09-03,70,2,0.3,5.1,35.0,24.0
3,2018-09-04,67,2,0.4,10.5,36.0,23.0
4,2018-09-05,66,2,0.4,11.3,36.0,23.0
...,...,...,...,...,...,...,...
115,2018-04-26,53,2,0.4,16.5,46.0,18.0
116,2018-04-27,29,1,0.5,12.9,43.0,10.0
117,2018-04-28,23,1,0.4,11.8,49.0,7.0
118,2018-04-29,34,1,0.3,6.0,49.0,17.0


In [134]:
df_agg_measures

NameError: name 'df_agg_measures' is not defined

In [86]:
df.columns

Index(['lat', 'lon', 'utc', 'parameter', 'concentration', 'unit',
       'raw_concentration', 'aqi', 'aqi_category', 'site_name', 'site_agency',
       'aqs_id', 'full_aqs_id', 'date'],
      dtype='object')

In [106]:
df_clean.shape

(11192, 14)

In [88]:
df[['date', 'aqi', 'aqi_category']]

,date,aqi,aqi_category
0,2018-09-01,26,1
1,2018-09-01,3,1
2,2018-09-01,-999,-999
3,2018-09-01,-999,-999
4,2018-09-01,32,1
...,...,...,...
11187,2018-12-31,-999,-999
11188,2018-12-31,7,1
11189,2018-12-31,6,1
11190,2018-12-31,-999,-999


In [113]:
def get_type(s):
    return s.dtype

df.apply(get_type, axis=0)

lat                  float64
lon                  float64
utc                   object
parameter             object
concentration        float64
unit                  object
raw_concentration    float64
aqi                    int64
aqi_category           int64
site_name             object
site_agency           object
aqs_id                 int64
full_aqs_id            int64
date                  object
dtype: object